Preparing data

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler

import implicit

In [2]:
df_train = pd.read_csv("../input/train.csv", low_memory=None, header=None)

In [3]:
df_train.columns = df_train.iloc[2713723]

In [4]:
df_train.drop(df_train.index[2713723], inplace=True)

In [5]:
df_train.drop(['deleted', 'deleter_fk'], axis=1, inplace=True)

In [6]:
df_train = df_train[~df_train['receipt_mfc'].str.match('^(?![0123456789])')]

In [7]:
df_train = df_train[~df_train['order_number'].str.match('^(?![0123456789])')]

In [8]:
df_train = df_train[~df_train['mfc'].str.match('^(?![0123456789])')]

In [28]:
df_train['service'] = df_train['service'].astype('int64')

In [45]:
df_train['cpgu_user'] = df_train['cpgu_user'].astype('int64')

In [51]:
df_train['mfc'] = df_train['mfc'].astype('int64')

In [60]:
df_train.drop(['receipt_mfc', 'creation_mfc', 'issue_date', 'close_date', 'id_x', 'id_y', 'id', 'cpgu_user'], axis=1, inplace=True)

In [64]:
def get_subdep(x):
    if x['sub_department'] is not np.nan:
        return 1
    return 0

In [65]:
df_train['is_subdep'] = df_train.apply(get_subdep, axis=1)

In [68]:
df_train.drop(['sub_department'], axis=1, inplace=True)

In [70]:
def get_csid(x):
    if x['custom_service_id'] is not np.nan:
        return 1
    return 0

In [71]:
df_train['is_csid'] = df_train.apply(get_csid, axis=1)

In [75]:
df_train['custom_service_id'].fillna(-1, inplace=True)

In [77]:
df_train['win_count'].fillna(df_train['win_count'].mean(), inplace=True)

In [79]:
df_train['order_date'] = df_train['order_date'].apply(pd.to_datetime)
df_train['change_timestamp'] = df_train['change_timestamp'].apply(pd.to_datetime)

In [108]:
df_train['proc_time'] = (df_train['change_timestamp']-df_train['order_date'])/np.timedelta64(1,'h')

In [112]:
def get_time(x):
    if x['proc_time'] < 0:
        return 0
    return x['proc_time']

In [113]:
df_train['proc_time'] = df_train.apply(get_time, axis=1)

In [116]:
df_train['dayofweek'] = df_train['order_date'].dt.dayofweek

In [127]:
def get_hour(x):
    h = x['order_date'].hour
    if h>=5 and h<11:
        return 'morning'
    elif h>=11 and h<17:
        return 'lunch'
    elif h>=17 and h<23:
        return 'evening'
    return 'night'

In [128]:
df_train['day_part'] = df_train.apply(get_hour, axis=1)

In [130]:
df_train.to_csv('clean_train.csv', index=False)

In [34]:
service = pd.read_csv("../input/cpgu_service.csv", sep=';', encoding='latin-1')

In [38]:
df_train = pd.merge(df_train, service[['id', 'person', 'sole', 'legal']], how='left', left_on = 'service', right_on = 'id')

In [42]:
cpgu_user = pd.read_csv("../input/cpgu_user.csv", sep=';')

In [46]:
df_train = pd.merge(df_train, cpgu_user, how='left', left_on = 'cpgu_user', right_on = 'id')

In [49]:
cpgu_mfc = pd.read_csv("../input/cpgu_mfc.csv", sep=';', encoding='latin-1')

In [53]:
df_train = pd.merge(df_train, cpgu_mfc[['id', 'win_count']], how='left', left_on = 'mfc', right_on = 'id')

In [ ]:
df_train.to_csv('clean_train.csv', index=False)